In [1]:
from collections import defaultdict
import copy
import random
import os
import shutil
from urllib.request import urlretrieve
import cv2
import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torchvision.models as models
import json
from torchvision import datasets, models, transforms
from PIL import Image
import numpy as np
import json
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

In [2]:

test_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        A.CenterCrop(height=256, width=256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)


In [3]:
with open('/content/classes.txt') as f:
    classes = f.readlines()
classes = [c.replace('\n','') for c in classes]

In [4]:
class2idx  = {c:i for i,c in enumerate(classes)}
idx2class  = {i:c for i,c in enumerate(classes)}

In [5]:
NCLASS = len(classes)

In [6]:

class ResNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = getattr(models, 'resnet18')(pretrained=True)
        #self.resnet.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        #for param in self.resnet.parameters():
        #    param.requires_grad = Tr
        ninput= self.resnet.fc.in_features
        self.resnet.fc = torch.nn.Linear(ninput,NCLASS)



    def forward(self, xb):
        #xb = xb.view(-1, 3, 224, 224) #xb = [bs=64,channel=1,height=244,width=244]
        xb = self.resnet(xb)   


        return xb

In [7]:
is_gpu = torch.cuda.is_available()


In [8]:
resnet =ResNet();

if is_gpu: 
    resnet.cuda()


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
if is_gpu: 
    resnet.load_state_dict(torch.load('/content/FSDL_backup.pt'))
else:
    device = torch.device('cpu')
    resnet.load_state_dict(torch.load('/content/FSDL_backup.pt', map_location=device))

In [10]:

with open('/content/food2ingredients.json', 'r') as f:
  food2ingredients = json.load(f)

In [11]:
def inference(img_path, model, transforms):
    model.eval()
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = transforms(image=image)["image"]
    outputs = model(image.unsqueeze(0))
    _, predicted = torch.max(outputs.data, 1)
    food = idx2class[predicted[0].item()]
    return {'food':food,'ingredients':food2ingredients[food]}


In [12]:
inference('/content/apple_pie.png',resnet,test_transforms)

{'food': 'apple_pie',
 'ingredients': 'sugar,nutmeg,milk,butter,flour,pastry,cinnamon,apples,lemon'}